In [ ]:
!pip install -r requirements.txt

In [ ]:
from LLM.llm import LLM

from conf.Tools import Tools
tools = Tools()

## 将App builder中获取的应用转换成标准的JSON schema格式，并对description进行优化

In [ ]:
import json
import os

os.environ['OPENAI_API_BASE'] = "https://fast.xeduapi.com/v1"
os.environ['OPENAI_API_KEY'] = "sk-jz0shLgMJY9HBVnLC3Fe3dCaA5204a418e67003f637f1eFf"
from tqdm import tqdm
function_data = tools.read_json("./datas/all_functions.json")

llm = LLM()
llm.setTemplate(temp=template, values=["description"])
genereated_json = list()
for item in tqdm(function_data):
    res = llm.run({"description": item})
    try:
        genereated_json.append(json.loads(res.replace('```json', '').replace('```', '')))
    except:
        genereated_json.append(res)

tools.write_2_json(genereated_json, "./datas/genereated_json.json")

## 生成输入输出的数据，每个Function先生成一个QA


In [ ]:
import json
import os

os.environ['OPENAI_API_BASE'] = "https://api.xeduapi.com"
os.environ['OPENAI_API_KEY'] = "sk-jz0shLgMJY9HBVnLC3Fe3dCaA5204a418e67003f637f1eFf"
from tqdm import tqdm

function_data = tools.read_json("./datas/genereated_json.json")

llm = LLM()
llm.setTemplate(temp=qa_generate_template, values=["function_intro"])
genereated_qa_json = list()
for item in tqdm(function_data):
    res = llm.run({"function_intro": item})
    try:
        genereated_qa_json.append(json.loads(res.replace('```json', '').replace('```', '')))
    except:
        genereated_qa_json.append(res)

tools.write_2_json(genereated_qa_json, "./datas/genereated_qa_json.json")

## 批量生成QA

In [ ]:
from LLM.llm import LLM

from conf.Tools import Tools
tools = Tools()

In [ ]:
def format_checker(function, input_parameters):
    assert function["name"] == input_parameters["function_name"]
    required_properties = function["parameters"]["required"]
    for k in required_properties:
        assert k in input_parameters["arguments"].keys()
    
    llm = LLM()
    llm.setTemplate(temp="""下面给你一个函数的描述，其中包括需要传入的参数信息。你需要判断给定的参数是否符合这个函数的参数要求，符合返回True，不符合返回False。

# 函数描述
{function}

# 给定的参数
{para}

# 给定的参数是否符合函数描述中的参数要求：""", values = ["function", "para"])
    llm.run({"function": function, "para": input_parameters})
    return True


In [ ]:
import json, sys
from tqdm import tqdm
import traceback

def generate_function_qa(item, llm):
    cur_function_qas = list()
    cur_function_qas.append(item)
    for i in tqdm(range(30)):
        tn = 5
        # while tn>0:
            # try:
        try:
            print([f"{qid}. {q['question']}" for qid,q in enumerate(cur_function_qas)])
        except:
            print(cur_function_qas)
        res = llm.run({"function_intro": item["function"], "questions": "\n".join([f"{qid}. {q['question']}" for qid,q in enumerate(cur_function_qas)])})
        ge_f = json.loads(res.replace('```json', '').replace('```', ''))
        if format_checker(ge_f["function"][0], ge_f["input_parameters"][0]):
            cur_function_qas.append(ge_f)
                # gn-=1
            # except Exception as e:
            #     print(f"line: {sys._getframe().f_lineno}\t {e} \ntraceback.print_exc(): {traceback.print_exc()}")
            #     tn-=1
            #     continue
    return cur_function_qas

In [ ]:
from conf.prompt import ICL_Generate_Prompt
import json, sys
qa_function_data = tools.read_json("/root/Projects/LLM-Application-base-framework/datas/genereated_qa_json1.json")

llm = LLM()
llm.setTemplate(temp=ICL_Generate_Prompt, values=["questions", "function_intro"])
all_function_res_qas = list()
run_paras = list()
for _id, item in enumerate(qa_function_data[:]):
    run_paras.append((item, llm))
    continue
    cur_function_qas = list()
    cur_function_qas.append(item)

    gn = 5
    while gn>0:
        print(gn)
        # try:
        res = llm.run({"function_intro": item["function"], "questions": "\n".join([f"{qid}. {q['question']}" for qid,q in enumerate(cur_function_qas)])})
        ge_f = json.loads(res.replace('```json', '').replace('```', ''))
        if format_checker(ge_f["function"][0], ge_f["input_parameters"][0]):
            cur_function_qas.append(ge_f)
            gn-=1
        # except Exception as e:
        #     print(f"line: {sys._getframe().f_lineno}\t {e}")
        #     print(res)
        #     continue
    
    all_function_res_qas.append(cur_function_qas)

all_function_res_qas = tools.multi_thread_run(12, generate_function_qa, run_paras, "Generate")
tools.write_2_json(all_function_res_qas, f"./datas/all_function_res_qas.json")


In [2]:
wf = open(f"datas/all_function_res_qas/0.jsonl", 'a+', encoding="utf-8")
wf.write("fffffffffffffffffffffffffffffffffffffff\n")
wf.close()